In [1]:
import pandas as pd
import csv
from Plan import CreateSamples_renewed
import importlib
import numpy as np

In [2]:
importlib.reload(CreateSamples_renewed)

<module 'Plan.CreateSamples_renewed' from 'C:\\Users\\Edwin\\Desktop\\OT2-DOE\\PlanPrepareProcess\\Plan\\CreateSamples_renewed.py'>

In [3]:
path = r"C:\Users\Edwin\Desktop\OT2-DOE\PlanPrepareProcess\04_28_21_Yttrium_Test.csv"
chem_path = r"Chemical Database.csv"

In [4]:
plan = CreateSamples_renewed.get_experiment_plan(path, chem_path)

In [5]:
concentration_df = CreateSamples_renewed.concentration_from_linspace(plan['Component Shorthand Names'], 
                                                                     plan['Component Concentration Linspaces [min, max, n]'],
                                                                     plan['Component Concentration Units'],
                                                                     unity_filter=False)

concentration_df

,Y(NO3)3 concentration molarity,KBF4 concentration molarity
0,0.015,0.03


In [6]:
complete_amounts_added = CreateSamples_renewed.determine_component_amounts(plan, concentration_df)
complete_amounts_added

You calculated for component masses given the provided units
You calculated for component masses given the provided units


,Y(NO3)3 concentration molarity,KBF4 concentration molarity,Y(NO3)3 amount mass g,Y(NO3)3 amount volume mL,KBF4 amount mass g,KBF4 amount volume mL
0,0.015,0.03,0.028726,0.010711,0.018885,0.007539


In [7]:
stock_dict = CreateSamples_renewed.stock_dictionary(plan['Stock Names'], plan['Stock Concentration Units'], plan['Stock Concentrations'])
# make it so when creating a stock_dict that if missing anything just raise an but still allow and fill missing with nan

In [8]:
complete_df = CreateSamples_renewed.calculate_stock_volumes_from_component_masses(plan, complete_amounts_added, stock_dict)
complete_df

,Y(NO3)3 concentration molarity,KBF4 concentration molarity,Y(NO3)3 amount mass g,Y(NO3)3 amount volume mL,KBF4 amount mass g,KBF4 amount volume mL,Y(NO3)3-water-stock amount volume mL,KBF4-water-stock amount volume mL
0,0.015,0.03,0.028726,0.010711,0.018885,0.007539,0.075,4.213483


In [9]:
complete_df = CreateSamples_renewed.calculate_common_solvent_residual_volumes(complete_df, stock_dict)
complete_df = CreateSamples_renewed.calculate_total_stock_volumes(complete_df)
complete_df.sum(axis=1)

0    8.687827
dtype: float64

In [10]:
def missing_volume(total_sample_volume, complete_df):
    stock_df = CreateSamples_renewed.isolate_common_column(complete_df, 'stock')
    total_stock_volume = stock_df.sum(axis=1)
    missing_volume = total_sample_volume-total_stock_volume
    complete_df['Missing Sample Volume mL'] = missing_volume
    return complete_df

In [11]:
complete_df = missing_volume(plan['Sample Amount'], complete_df)
complete_df

,Y(NO3)3 concentration molarity,KBF4 concentration molarity,Y(NO3)3 amount mass g,Y(NO3)3 amount volume mL,KBF4 amount mass g,KBF4 amount volume mL,Y(NO3)3-water-stock amount volume mL,KBF4-water-stock amount volume mL,Total Volume mL,Missing Sample Volume mL
0,0.015,0.03,0.028726,0.010711,0.018885,0.007539,0.075,4.213483,4.288483,0.711517


In [ ]:
# so when making all stocks at the end the mixtures need to make their own stock, 
# very easy just at the end of the checking for common stocks, iterate through all the mixture labels 
# and if the solvent if resent in the pure ones then move on, if not then make an entry.
# but would this mess up something else since you are iterating sometimes through the stock_dict

In [19]:
def complete_missing_volume_with_commmon_solvent(complete_df, stock_dict, solvent=None):

    stock_volumes = CreateSamples_renewed.isolate_common_column(complete_df, 'stock')
    stock_names = [CreateSamples_renewed.identify_component_name(col) for col in stock_volumes if 'stock' in col] # this is seperate from the dict since your dict can have more stocks just need ot have the same name
    pure_common_solvent_stocks = [stock_name for stock_name in stock_names if stock_dict[stock_name]['Common Solvent'] == 'Pure']
    mixture_common_solvent_stocks = [stock_name for stock_name in stock_names if stock_dict[stock_name]['Common Solvent'] == 'Mixture']
    assert not len(pure_common_solvent_stocks) > 1, 'Too many common solvents, select one.'
    
    
    if solvent:
        solvent = solvent
    elif len(pure_common_solvent_stocks) == 1:
        # then select one and compelte/relpace the missing sampel volume and then if the solvent is provided then that superceeds
        solvent = next(iter(pure_common_solvent_stocks))
    print(solvent)
    complete_df[solvent + ' stock volume mL']

    return complete_df

In [21]:
stock_dict

{'Y(NO3)3-water-stock': {'solutes': ['Y(NO3)3'],
  'solvents': 'water',
  'unit': 'molarity',
  'concentration': 1,
  'Density (g/mL)': nan,
  'Common Solvent': 'Mixture'},
 'KBF4-water-stock': {'solutes': ['KBF4'],
  'solvents': 'water',
  'unit': 'molarity',
  'concentration': 0.0356,
  'Density (g/mL)': nan,
  'Common Solvent': 'Mixture'}}

In [20]:
complete_df = complete_missing_volume_with_commmon_solvent(complete_df, stock_dict)

None


TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [ ]:
total_sample_unit = plan['Sample Unit']
total_sample_amount = plan['Sample Amount']
if total_sample_unit = 'mL' : # the completing things of volume will only be for mgpermL if using wtf or volf then should be completeed from unity filter, but no necesarrily can also be done down here
    if total_sample_amount < complete_df['Total Volume mL']:
        print('do')